In [1]:
import warnings
warnings.filterwarnings('ignore')

In [29]:
def get_home_stats(comp, year, home_team, away_team):
    
    from scores_and_fixtures import season_results_and_fixtures
    import check_name
    from get_position import get_basic_position, get_detailed_position
    from datetime import datetime
    import pandas as pd
    import requests
    from bs4 import BeautifulSoup

    results_df = season_results_and_fixtures(comp, year)
    home_team = check_name.get_team_name(home_team)
    away_team = check_name.get_team_name(away_team)
    
    for i, row in results_df.iterrows():
        if row['home_team'] == home_team and row['away_team'] == away_team:
            url = f"https://fbref.com/{row['match_url']}"
            resp = requests.get(url) 
            resp.encoding = 'utf-8'
            soup = BeautifulSoup(resp.text.replace('<!--', '').replace('--!>', ''), 'html.parser')
            
            div_lineup_home = soup.find('div', {'class':'lineup'}, {'id':'a'})
            numbers, player_ids, player_names, positions = [], [], [], []
            
            players = div_lineup_home.find_all('td')
                
            for i, value in enumerate(players):
                if i % 2 == 0:
                    numbers.append(value.get_text())
                elif i % 2 == 1: 
                    player_names.append(value.get_text())
                    player_ids.append(value.a.get('href').split('/')[3])
                    
            df = pd.DataFrame(list(zip(numbers, player_ids, player_names)), columns = ['squad_number', 'player_id', 'player'])
            
            for i, row in df.iterrows():
                try:
                    player_div = soup.find("div", {"title":{row['player']}})
                    style = player_div["style"]
                    get_position = get_basic_position(style)
                    df.at[i, 'position'] = get_position
                    positions.append(get_position)
                except: pass
                
            keepers = positions.count("GK")
            defenders = positions.count("DF")
            midfielders = positions.count("MF")
            forwards = positions.count("FW")
                
            for i, row in df.iterrows():
                try:
                    player_div = soup.find("div", {"title":{row['player']}})
                    style = player_div["style"]
                    get_det_position = get_detailed_position(style, keepers, defenders, midfielders, forwards)
                    df.at[i, 'det_pos'] = get_det_position
                except: pass
                
            df = df.fillna('Sub')
            
    print(url)

    return df

In [30]:
get_home_stats("esp", 2023, "real madrid", "sevilla")

https://fbref.com//en/matches/0788c1f1/Real-Madrid-Sevilla-February-25-2024-La-Liga


,squad_number,player_id,player,position,det_pos
0,13,3061e1ec,Andriy Lunin,GK,GK
1,6,2946f9a1,Nacho,DF,CB
2,7,7111d552,Vinicius Júnior,FW,WF
3,8,6ce1f46f,Toni Kroos,MF,CM
4,11,8f5e92a6,Rodrygo,FW,ST
5,15,0959c2a2,Federico Valverde,MF,CM
6,17,fd51b456,Lucas Vázquez,DF,WB
7,18,4f255115,Aurélien Tchouaméni,MF,CM
8,21,407feb71,Brahim Díaz,FW,WF
9,22,18b896d6,Antonio Rüdiger,DF,CB
